In [442]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [443]:
from bs4 import BeautifulSoup
import re
from uuid import uuid4

In [444]:
class Crawler:

    def get_html(self):
        pass

    def get_card_info(self):
        pass

    def get_links(self, page):
        pass

In [445]:
class CrawlerEda(Crawler):

    def get_html(self, page):
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-popup-blocking")

        browser = webdriver.Chrome(options=chrome_options)
        url = f"https://eda.ru/recepty?page={page}"
        #browser.implicitly_wait(30)
        browser.get(url)
        wait = WebDriverWait(browser, 10)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        return soup

    def get_card_info(self, number):
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-popup-blocking")

        browser = webdriver.Chrome(options=chrome_options)
        action = ActionChains(browser)
        count = 0

        #linker = linkers[number]['link']
        #browser.get(linker)
        wait = WebDriverWait(browser, 10)
        soup_card = BeautifulSoup(browser.page_source, 'html.parser')
        return soup_card

    def get_links(self, page):
        links = []
        for i in range(1, page):
            html = CrawlerEda().get_html(i)
            links += ParserEda(html).get_cards()
        return links

In [446]:
class Parser:

    def __init__(self, html):
        self.html = html

    def get_cards(self):
        pass

In [447]:
class ParserEda(Parser):

    def get_cards(self):
        for recipe in self.html.findAll('div', class_='emotion-1f6ych6'):
            try:
                link = "https://eda.ru"+recipe.find('a', class_='emotion-12sjte8').get('href')
                photo = "https://eda.ru"+recipe.find('img', class_='emotion-1bi124x').get('src')
                print(link)
            except AttributeError:
                print("There's no item with that code")
        return Link(0, link, photo)


In [448]:
class Recipe:
    def __init__(self, key, name_dish, count_portion, cook_time, ingredients, manual):
        self.id = key
        self.name_dish = name_dish
        self.count_portion = count_portion
        self.cook_time = cook_time
        self.ingredients = ingredients
        self.manual = manual

In [449]:
class Ingredient:
    def __init__(self, key, name_ingr):
        self.id = key
        self.name_ingr = name_ingr

In [450]:
class Calories:
    def __init__(self, key, calories, proteins, fats, carbohydrates):
        self.id = key
        self.calories = calories
        self.proteins = proteins
        self.fats = fats
        self.carbohydrates = carbohydrates

In [451]:
class Link:
    def __init__(self, key, link, photo):
        self.id = key
        self.link = link
        self.photo = photo

In [452]:
class DBHelper:

    def __init__(self, db_name = "recipe.db"):
        self.dbname = f"sqlite3:/{db_name}"
        self.conn = sqlite3.connect(db_name)

    def simple_create_table(self, table_name, primary_key, params):
        for i in len(params) - 1:
            column = params[i] + " TEXT, "
        column += params[len(params)] + " TEXT "
        new_table = f"CREATE TABLE IF NOT EXISTS {table_name} ({primary_key} TEXT PRIMARY KEY , {column})"
        self.conn.execute(new_table)
        self.conn.commit()

    def create_table(self, table_name, primary_key, foreign_table_name, foreign_col, another_foreign_col, params):
        for i in len(params) - 1:
            column = params[i] + " TEXT, "
        column += params[len(params)] + " TEXT "

        foreign_key = f"FOREIGN KEY ({foreign_col})  REFERENCES {foreign_table_name} ({another_foreign_col})"

        new_table = f"CREATE TABLE IF NOT EXISTS {table_name} ({primary_key} TEXT PRIMARY KEY , {column}, {foreign_key})"
        self.conn.execute(new_table)
        self.conn.commit()

    def insert(self, table_name, args):
        for i in len(args) - 1:
            arguments = args[i] + ", "
        arguments += args[len(args)]

        new_insert = f"INSERT INTO {table_name} VALUES ({arguments})"
        self.conn.execute(new_insert)
        self.conn.commit()

    #def print_info(self):


    #def update(self, flat_id, updates):


    #def delete(self, flat_id):


In [453]:
links = CrawlerEda().get_links(2)

In [455]:

linkers = []
for i in range(1, 2):
    html = CrawlerEda().get_html(i)
    linkers += ParserEda(html).get_cards()

In [458]:
html = CrawlerEda().get_html(1)
print(html)